In [63]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, html, datetime
import openpyxl
from openpyxl import Workbook 
from os import listdir
from os.path import isfile, join
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator   
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')

In [64]:
pl=sql_todf("""
SELECT UPPER(LTRIM(RTRIM(DisplayValue))) [Selections], PK_PickList [ML_Value], PK_PickListType [PLT]
FROM vwPickLists  
WHERE PK_PickListType > 410
ORDER BY PK_PickListType DESC
""", connstr)  
s=list(pl.loc[pl['PLT']==427]['Selections'])
print(s)

['YES, INTERNAL TRAFFIC ONLY', 'YES, EXTERNAL TRAFFIC ONLY', 'YES, BOTH INTERNAL AND EXTERNAL TRAFFIC', 'NO', 'UNKNOWN']


In [65]:
filepath = r'C:\Users\timko\Downloads\Questionnaire\HVA Data Protection Questionnaire v0.1.xlsx'
path=r'C:\Users\timko\Downloads\Questionnaire\\'
files = [f for f in listdir(path) if re.search('^HVA.*1\.xlsx', f)]
print(files)
xlfile=files[2]
xlsx= openpyxl.load_workbook(f'{path}{xlfile}')
xlfile
 

['HVA Asset Management Questionnaire v0.1.xlsx', 'HVA Business Continuity Management Questionnaire v0.1.xlsx', 'HVA Data Protection Questionnaire v0.1.xlsx', 'HVA Endpoint Protection Questionnaire v0.1.xlsx', 'HVA Identity and Access Management Questionnaire v0.1.xlsx', 'HVA Monitoring and Detection Questionnaire v0.1.xlsx', 'HVA Network Architecture Questionnaire v0.1.xlsx', 'HVA Remote Access Questionnaire v0.1.xlsx', 'HVA Vulnerability Management Questionnaire v0.1.xlsx']


'HVA Data Protection Questionnaire v0.1.xlsx'

In [66]:
converts={
    'Selections':lambda x: re.sub('\d{,2}\.','',str(x)).upper().strip()
    , 'ML':lambda x: str(x).strip()
    , 'SML':lambda x: re.sub('[^A-Z0-9]','',str(x)).strip()  }
df=pd.read_excel(f'{path}{xlfile}', converters=converts,  sheet_name=4, header=2, usecols='B:I').reset_index()
df = df.rename(columns={'ML Score': 'ML_Score','ML Scored': 'ML_Scored' })
for c in ['ID','Question','SML']:
    df[c] = df[c].replace(to_replace=[0, '0'], method='ffill')
df["ID"].fillna(method='ffill', inplace=True) 
df.drop(columns=['Check Answer', 'index'], inplace=True)
df.loc[df['ID']=='DP3']

,ID,Question,Selections,SML,ML,ML_Score,ML_Scored
8,DP3,Does the agency have established HVA data prot...,YES,SML1,1,1,0
9,DP3,Does the agency have established HVA data prot...,NO,SML1,1,0,0
10,DP3,Does the agency have established HVA data prot...,UNKNOWN,SML1,1,0.001,0


In [67]:
df = df.loc[ df["ID"].str.contains('^\w\w\d(\w){0,5}$', na=False, regex=True) ]  
df = df.loc[ df["ML"].str.contains('\d', na=False,  regex=True, case=False) ]  
df['ML_Score'] = df['ML_Score'].apply(lambda s: re.sub('[^0-9\.]','',str(s)))
df['Selections'] = df['Selections'].apply(lambda x: re.sub('[^A-Za-z0-9\s]','',str(x)))
df.loc[df['ID']=='DP3']

,ID,Question,Selections,SML,ML,ML_Score,ML_Scored
8,DP3,Does the agency have established HVA data prot...,YES,SML1,1,1,0
9,DP3,Does the agency have established HVA data prot...,NO,SML1,1,0,0
10,DP3,Does the agency have established HVA data prot...,UNKNOWN,SML1,1,0.001,0


In [68]:
dfp=df.groupby(['ID'], as_index=False).agg({'Selections':list}) 
pr = picklist_recommender(ctx.config['connstr'], reset_cache=True, picklist_where=' PK_PickListType > 410 ') 
dfp['PLT']=dfp['Selections'].apply(lambda s: pr.recommend(s)['PK_PicklistType'])
pr.to_cache() 

dff=pd.merge(df,dfp.loc[:,['ID','PLT']], how='left', left_on='ID', right_on='ID')
dff=pd.merge(dff,pl, how='left', left_on=['Selections','PLT'], right_on=['Selections','PLT'])
dff.ML_Value=dff.ML_Value.fillna(0).astype('int32').astype('str')
dff.loc[dff['ID']=='DP3']

,ID,Question,Selections,SML,ML,ML_Score,ML_Scored,PLT,ML_Value
0,DP3,Does the agency have established HVA data prot...,YES,SML1,1,1,0,423,4714
1,DP3,Does the agency have established HVA data prot...,NO,SML1,1,0,0,423,4715
2,DP3,Does the agency have established HVA data prot...,UNKNOWN,SML1,1,0.001,0,423,4716


In [69]:
df1=dff
df1.SML.fillna(method='ffill', inplace=True)
df1=df1.loc[df1['ML_Score'] != '0']
df1['sortorder'] = range(1, len(df1)+1)
#df1['ML_Score']=df1['ML_Score'].astype(float)
sql=SQL_INSERT_FROM_DF(df1.loc[:, ['ID', 'SML', 'ML','ML_Score', 'ML_Scored', 'ML_Value', 'sortorder']])
with open(f'{ctx.get_dest()}script.sql', 'w') as f:
    f.write('\n'.join(sql))
df1.loc[df1['ID']=='DP3']

--DECLARE @T AS TABLE (ID NVARCHAR(MAX), SML NVARCHAR(MAX), ML NVARCHAR(MAX), ML_Score NVARCHAR(MAX), ML_Scored NVARCHAR(MAX), ML_Value NVARCHAR(MAX), sortorder NVARCHAR(MAX))
INSERT INTO @T (ID, SML, ML, ML_Score, ML_Scored, ML_Value, sortorder) VALUES ('DP3', 'SML1', '1', '1', 0, '4714', 1);
INSERT INTO @T (ID, SML, ML, ML_Score, ML_Scored, ML_Value, sortorder) VALUES ('DP3', 'SML1', '1', '0.001', 0, '4716', 2);
INSERT INTO @T (ID, SML, ML, ML_Score, ML_Scored, ML_Value, sortorder) VALUES ('DP3a', 'SML1', '1', '1', 0, '4714', 3);
INSERT INTO @T (ID, SML, ML, ML_Score, ML_Scored, ML_Value, sortorder) VALUES ('DP3a', 'SML1', '1', '0.001', 0, '4716', 4);
INSERT INTO @T (ID, SML, ML, ML_Score, ML_Scored, ML_Value, sortorder) VALUES ('DP3b', 'SML1', '1', '1', 0, '4714', 5);
INSERT INTO @T (ID, SML, ML, ML_Score, ML_Scored, ML_Value, sortorder) VALUES ('DP3b', 'SML1', '1', '0.001', 0, '4716', 6);
INSERT INTO @T (ID, SML, ML, ML_Score, ML_Scored, ML_Value, sortorder) VALUES ('DP3c', 'SML1',

,ID,Question,Selections,SML,ML,ML_Score,ML_Scored,PLT,ML_Value,sortorder
0,DP3,Does the agency have established HVA data prot...,YES,SML1,1,1,0,423,4714,1
2,DP3,Does the agency have established HVA data prot...,UNKNOWN,SML1,1,0.001,0,423,4716,2
